In [1]:
import joblib

In [2]:
import numpy as np

In [3]:
import os

In [6]:
import processing

/home/gito/.local/lib/python3.6/site-packages/numba/core/errors.py:154: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


# Load data

In [7]:
DATA_PATH = './data'

In [8]:
with open('{}/splits/crossmodal_train.txt'.format(DATA_PATH), 'r') as f:
    train_splits = f.readlines()
    
train_splits = [i.replace('\n', '') for i in train_splits]

with open('{}/splits/crossmodal_test.txt'.format(DATA_PATH), 'r') as f:
    test_splits = f.readlines()
    
test_splits = [i.replace('\n', '') for i in test_splits]

with open('{}/splits/crossmodal_val.txt'.format(DATA_PATH), 'r') as f:
    val_splits = f.readlines()
    
val_splits = [i.replace('\n', '') for i in val_splits]

In [9]:
poses = ['{}/motions/'.format(DATA_PATH) + i for i in os.listdir('{}/motions/'.format(DATA_PATH))]

In [10]:
music_path = ['{}/music/'.format(DATA_PATH) + i for i in os.listdir('{}/music/'.format(DATA_PATH))]

In [11]:
poses_unflat = {}

In [12]:
# Convert from joint3d to rotation matrix
# Note: default matrix should be
# [[0, -r[2], r[1]],
# [r[2], 0, -r[0]],
# [-r[1], r[0], 0]]
# But its generates nested elements, that is not okay for further processing

def Rodrigues(rotvec):
    theta = np.linalg.norm(rotvec)
    r = (rotvec/theta).reshape(3, 1) if theta > 0. else rotvec
    cost = np.cos(theta)
    mat = np.asarray([[[0], -r[2], r[1]],
                      [r[2], [0], -r[0]],
                      [-r[1], r[0], [0]]])
    tmp = (cost*np.eye(3) + (1-cost)*r.dot(r.T) + np.sin(theta)*mat)
    return tmp

In [18]:
# def Rodrigues1(rotvec):
#     theta = np.linalg.norm(rotvec)
#     r = (rotvec/theta).reshape(3, 1) if theta > 0. else rotvec
#     cost = np.cos(theta)
#     mat = np.asarray([[0, -r[2], r[1]],
#                       [r[2], 0, -r[0]],
#                       [-r[1], r[0], 0]])
#     tmp = (cost*np.eye(3) + (1-cost)*r.dot(r.T) + np.sin(theta)*mat)
#     return tmp

In [13]:
for path in poses:
    try:
        output = joblib.load(path)# ['smpl_poses']
        poses = output['smpl_poses'].reshape(output['smpl_poses'].shape[0], 24, 3)
        poses = np.array([[[val[:, j] for j, val in enumerate(Rodrigues(i))] for i in pose] for pose in poses])

        # flatten all poses
        poses = poses.reshape(output['smpl_poses'].shape[0], 24 * 9)
        trace = output['smpl_trans']

        if len(np.where(poses > 5)[0]) > 0:
            continue
        else:
            poses_unflat[path.split('/')[-1]] = np.concatenate([poses, trace], 1)
    except KeyboardInterrupt:
        raise
    except Exception as e:
        print(path, e)

KeyboardInterrupt: 

In [15]:
# load music files and process them using aist++ utils
music = {}

for path in music_path:
    features = processing.music_features_all(path)
#     features = np.concatenate([features[i] if i not in ['peak_onehot', 'beat_onehot'] else features[i][:, None] for i in features.keys()], -1)
    music[path.split('/')[-1][:-4]] = features

KeyboardInterrupt: 

In [17]:
music_normalization = {}
for fet in music['mJS5'].keys():
    music_normalization[fet] = {}
    music_normalization[fet]['min'] = min([music[i][fet].astype(float).min() for i in music.keys()])
    music_normalization[fet]['max'] = max([(music[i][fet].astype(float) - music_normalization[fet]['min']).max() for i in music.keys()])

KeyError: 'mJS5'

In [18]:
params = list(music_normalization.keys())

In [19]:
music_norm = {}

for key, samp in music.items():
    features = np.concatenate([samp[i]
                               if i not in ['peak_onehot', 'beat_onehot'] 
                               else samp[i][:, None]
                               for i in params], -1)
    music_norm[key] = features

ValueError: need at least one array to concatenate

In [47]:
real_samples_dataset_180 = []
real_samples_dataset_300 = []

In [48]:
# we split sequences into 300-length subsequences
for key, value in poses_unflat.items():
    music_related = music_norm[key.split('_')[-2]]

    if key.split('.')[0] in train_splits:
        types = 'train'
    elif key.split('.')[0] in test_splits:
        types = 'test'
    elif key.split('.')[0] in val_splits:
        types = 'val'
    else:
        types = 'broken'
        
    for split in np.array(range(value.shape[0]))[::300]:
        tmp = {}

        if value[split:split+180].shape[0] == 180:
            tmp['type'] = types
            tmp['pose'] = value[split:split+180]
            tmp['music'] = music_related[split:split+180]
            real_samples_dataset_180.append(tmp)

#             tmp = value[split:split+300]
        tmp = {}

        if value[split:split+300].shape[0] == 300:
            tmp['type'] = types
            tmp['pose'] = value[split:split+300]
            tmp['music'] = music_related[split:split+300]
            real_samples_dataset_300.append(tmp)

In [57]:
real_samples_dataset_300 = np.array(real_samples_dataset_300)

In [58]:
real_samples_dataset_180 = np.array(real_samples_dataset_180)

In [60]:
np.save('{}/processed/rotation_matrix.300'.format(DATA_PATH), real_samples_dataset_300)

In [61]:
np.save('{}/processed/rotation_matrix.180'.format(DATA_PATH), real_samples_dataset_180)